In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 # Updated data for the new table provided

    updated_data = {
    "LLM's": ["GPT 3.5", "Llama2-7B", "Mistral-7B", "Falcon-7B", "MedAlpaca"],
    "Average BLEU": [0.024, 0.020, 0.017, 0.030, 0.002],
    "Average ROUGE-1": [0.39, 0.35, 0.34, 0.19, 0.15],
    "Average ROUGE-2": [0.12, 0.10, 0.095, 0.032, 0.036],
    "Average ROUGE-L": [0.20, 0.186, 0.185, 0.12, 0.097],
    "Average METEOR": [0.26, 0.24, 0.23, 0.11, 0.09]
}

# Create an updated DataFrame
updated_df = pd.DataFrame(updated_data)

# Plotting the updated data
fig, ax = plt.subplots(figsize=(10, 6))
updated_df.set_index("LLM's").plot(kind='bar', ax=ax)

# Setting the title and labels for the updated plot
ax.set_title('Comparison of LLMs across Various NLP Metrics', fontsize=15)
ax.set_xlabel('LLMs', fontsize=12)
ax.set_ylabel('Values', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title="Metrics")

# Save the updated plot to a file
file_path = '/mnt/data/updated_model_comparison_chart.png'
fig.savefig(file_path)

plt.show()

file_path


In [ ]:
import pandas as pd

# Load the Excel file
excel_path = '/mnt/data/combined_file.xlsx'
xl = pd.ExcelFile(excel_path)

# Read all sheets into a dictionary of DataFrames
sheets_dict = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

sheets_dict.keys(), {sheet: df.shape for sheet, df in sheets_dict.items()}  # Display sheet names and their dimensions


In [ ]:
# Correcting the model name from 'GPT3' to 'GPT 3.5' in the DataFrame
combined_for_plot['model'] = combined_for_plot['model'].replace({'GPT3': 'GPT 3.5'})

# Recalculate the correlations with the corrected model name
corrected_correlations = {}
for model in combined_for_plot['model'].unique():
    model_df = combined_for_plot[combined_for_plot['model'] == model]
    corrected_correlations[model] = model_df['article_length'].corr(model_df['meteor_scores'])

# Now replot the correlations for each model with the corrected name
g = sns.FacetGrid(combined_for_plot, col="model", col_wrap=3, height=4, sharex=False, sharey=False)
g = g.map(sns.scatterplot, "article_length", "meteor_scores")

# Add a title and adjust the layout
g.fig.suptitle('Correlation between Article Length and METEOR Scores for Each Model', y=1.03)
g.fig.tight_layout()

# Show the plot
plt.show()

# Display the recalculated correlations
corrected_correlations


In [ ]:
# Reload the data since the state was reset
xl = pd.ExcelFile('/mnt/data/combined_file.xlsx')
sheets_dict = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

# Update the model names and calculate the summary length and article length for each sheet
for sheet_name, df in sheets_dict.items():
    df['model'] = sheet_name.replace('output', '').replace('_', ' ').strip()
    df['article_length'] = df['articles'].apply(len)
    df['summary_length'] = df['model_generated'].apply(len)

# Combine all sheets into one DataFrame for visualization
combined_df = pd.concat(sheets_dict.values(), ignore_index=True)

# Histograms of METEOR Scores
plt.figure(figsize=(10, 6))
for model in combined_df['model'].unique():
    sns.histplot(combined_df[combined_df['model'] == model]['meteor_scores'], kde=True, label=model)
plt.title('Distribution of METEOR Scores for Each Model')
plt.xlabel('METEOR Score')
plt.ylabel('Count')
plt.legend()
plt.show()

# Line Plot Over Articles
plt.figure(figsize=(10, 6))
for model in combined_df['model'].unique():
    model_df = combined_df[combined_df['model'] == model].sort_index()
    plt.plot(model_df.index, model_df['meteor_scores'], label=model)
plt.title('METEOR Scores Over Articles for Each Model')
plt.xlabel('Article Index')
plt.ylabel('METEOR Score')
plt.legend()
plt.show()

# Heatmap of Summary Lengths and METEOR Scores
heatmap_data = combined_df.pivot_table(index='summary_length', columns='model', values='meteor_scores', aggfunc='mean')
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap='viridis')
plt.title('Heatmap of Average METEOR Score by Summary Length for Each Model')
plt.xlabel('Model')
plt.ylabel('Summary Length')
plt.show()

# Pair Plot
sns.pairplot(combined_df[['article_length', 'summary_length', 'meteor_scores', 'model']], hue='model', plot_kws={'alpha':0.5})
plt.show()

# Bar Plot of Average Scores
avg_scores = combined_df.groupby('model')['meteor_scores'].mean().sort_values()
plt.figure(figsize=(10, 6))
avg_scores.plot(kind='bar')
plt.title('Average METEOR Scores for Each Model')
plt.xlabel('Model')
plt.ylabel('Average METEOR Score')
plt.show()

# Violin Plots
plt.figure(figsize=(10, 6))
sns.violinplot(x='model', y='meteor_scores', data=combined_df)
plt.title('Violin Plot of METEOR Scores for Each Model')
plt.xlabel('Model')
plt.ylabel('METEOR Score')
plt.show()


In [ ]:
import pandas as pd

# Loading the Excel file
file_path = '/mnt/data/combined_file.xlsx'

# Reading the file to check the sheet names
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
sheet_names


In [ ]:
# Extracting METEOR scores using the correct column name
meteor_scores_corrected = {}
for sheet in sheet_names:
    data = pd.read_excel(file_path, sheet_name=sheet)
    meteor_scores_corrected[sheet] = data['meteor_scores']

# Creating the box plot with the correct METEOR scores
plt.figure(figsize=(10, 6))
plt.boxplot(meteor_scores_corrected.values(), labels=meteor_scores_corrected.keys(), boxprops=dict(color="green"))
plt.title('METEOR Scores of Different Models')
plt.ylabel('METEOR Score')
plt.xticks(rotation=45)
plt.grid(True)
plt.gca().set_facecolor('#d4edd9')
plt.show()


In [ ]:
# Recreating the box plot with filled green color for the boxes
plt.figure(figsize=(10, 6))
plt.boxplot(meteor_scores_corrected.values(), labels=meteor_scores_corrected.keys(), 
            boxprops=dict(facecolor="green", color="green"), whiskerprops=dict(color="green"),
            capprops=dict(color="green"), medianprops=dict(color="green"), patch_artist=True)
plt.title('METEOR Scores of Different Models')
plt.ylabel('METEOR Score')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
# Loading the uploaded Excel file containing hallucination scores
hallucination_file_path = '/mnt/data/NER exact hallucination.xlsx'

# Reading the file into a DataFrame
hallucination_scores_df = pd.read_excel(hallucination_file_path)

# Displaying the first few rows of the DataFrame to understand its structure
hallucination_scores_df.head()


In [ ]:
# Creating box plots for each model to observe the distribution of hallucination scores
plt.figure(figsize=(12, 6))
hallucination_scores_df.boxplot()
plt.title('Distribution of Hallucination Scores for Each Model')
plt.ylabel('Hallucination Score')
plt.xlabel('Model')
plt.grid(True)
plt.show()

# Creating histograms for each model
plt.figure(figsize=(12, 8))
for i, model in enumerate(hallucination_scores_df.columns):
    plt.subplot(3, 2, i + 1)
    hallucination_scores_df[model].hist(bins=20, alpha=0.7)
    plt.title(model)
    plt.xlabel('Hallucination Score')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Creating density plots for each model
plt.figure(figsize=(12, 6))
for model in hallucination_scores_df.columns:
    hallucination_scores_df[model].plot(kind='density', label=model, linewidth=2, alpha=0.7)
plt.title('Density of Hallucination Scores for Each Model')
plt.xlabel('Hallucination Score')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Re-loading the uploaded Excel file as the previous session state was reset
hallucination_file_path = '/mnt/data/NER exact hallucination.xlsx'
hallucination_scores_df = pd.read_excel(hallucination_file_path)

# Renaming 'GPT3' to 'GPT 3.5' in the DataFrame
hallucination_scores_df_renamed = hallucination_scores_df.rename(columns={'GPT3': 'GPT 3.5'})

# Recreating the visualizations with the updated column name

# Box Plots
plt.figure(figsize=(12, 6))
hallucination_scores_df_renamed.boxplot()
plt.title('Distribution of Hallucination Scores for Each Model')
plt.ylabel('Hallucination Score')
plt.xlabel('Model')
plt.grid(True)
plt.show()

# Histograms
plt.figure(figsize=(12, 8))
for i, model in enumerate(hallucination_scores_df_renamed.columns):
    plt.subplot(3, 2, i + 1)
    hallucination_scores_df_renamed[model].hist(bins=20, alpha=0.7)
    plt.title(model)
    plt.xlabel('Hallucination Score')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Density Plots
plt.figure(figsize=(12, 6))
for model in hallucination_scores_df_renamed.columns:
    hallucination_scores_df_renamed[model].plot(kind='density', label=model, linewidth=2, alpha=0.7)
plt.title('Density of Hallucination Scores for Each Model')
plt.xlabel('Hallucination Score')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Recreating the density plots with distinct colors for each model

colors = ['blue', 'red', 'green', 'purple', 'orange']  # Defining a list of colors for distinction

plt.figure(figsize=(12, 6))
for model, color in zip(hallucination_scores_df_renamed.columns, colors):
    hallucination_scores_df_renamed[model].plot(kind='density', label=model, linewidth=2, color=color)
plt.title('Density of Hallucination Scores for Each Model')
plt.xlabel('Hallucination Score')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()
